In [1]:
import matplotlib.pyplot as plt
import numpy as np
import cv2
import os
from PIL import Image
import matplotlib.pyplot as plt
import random
import pathlib
import pandas as pd
import torch
from torchvision import transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from torchvision.io import read_image
from torch.utils.data import Dataset
from torchvision import transforms
from utilss import get_accuracy, plot_curve, keep_store_dict, store_dict_to_disk
from data_loader import ImageDataset
from initializers import gaussian_initializer, constant_initializer
from models import Model3, ModelM2, ModelM3, Model2, Model1, ModelAVE, Model
from test import test
from train import train

In [2]:
model1 = Model1()
model2 = Model2()
modelAVE = ModelAVE()
model2_M = ModelM2()
model3 = Model3()
model3_M = ModelM3()

In [3]:
model = Model(model1, model2, model3, modelAVE, model2_M, model3_M)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)

In [4]:
dataset_test = ImageDataset("heat_maps_test_plain", "images_test", (368, 368), model.center_map)
dataset_training = ImageDataset("heat_maps_training_plain", "images_training", (368, 368), model.center_map)
dataset_valid = ImageDataset("heat_maps_valid_plain", "images_valid", (368, 368), model.center_map)

In [5]:
loss_function = nn.MSELoss()
learning_rate = 0.001
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [6]:
num_epoch = 3
batch_size = 8
store_dict = dict()

In [ ]:
store_dict = train(model=model, num_epochs=num_epoch, train_loader=dataset_training, test_loader=dataset_valid, store_dict=store_dict, device = device, loss_function = loss_function, optimizer=optimizer, batch_size = batch_size)
store_dict_to_disk('store_dict.json', store_dict=store_dict)

0it [00:00, ?it/s]/usr/local/lib/python3.11/dist-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(
8001it [36:03,  3.70it/s]


Epoch: 1 | Loss: 0.1799 | Train Accuracy: 0.10


10it [00:02,  3.56it/s]


test acc = 0.1823529411764706


7348it [33:26,  3.20it/s]

In [ ]:
torch.save(model,'/notebooks/lk-s-2024-detekcija-poze/model22.7.pth')

In [ ]:
loaded_model = torch.load('model22.7.pth')
loaded_model.eval()

input1 = dataset_test[0][0]
input1 = input1.to(device)
output = loaded_model(input1)
output = output.cpu()
numpy_array = output.detach().numpy()
plt.imshow(numpy_array[0]*255)
plt.show()